In [6]:
import pandas as pd
import os

In [249]:
mv_data = pd.read_csv("/Users/yhschan/Desktop/sports analytics/Basketball-case-study-by-HMM/basketball/data/CHA-TOR.csv")

In [250]:
mv_data

,team_id,player_id,x_loc,y_loc,radius,game_clock,shot_clock,quarter,game_id,event_id,x_loc_original,y_loc_original
0,-1,-1,-201.2734,186.7540,3.64299,711.26,11.99,1,21500492,1,23.42540,45.12734
1,1610612761,2449,110.8853,143.3811,0.00000,711.26,11.99,1,21500492,1,19.08811,13.91147
2,1610612761,201960,114.5297,53.6935,0.00000,711.26,11.99,1,21500492,1,10.11935,13.54703
3,1610612761,200768,-195.1006,160.6838,0.00000,711.26,11.99,1,21500492,1,20.81838,44.51006
4,1610612761,201942,-137.9570,94.1535,0.00000,711.26,11.99,1,21500492,1,14.16535,38.79570
...,...,...,...,...,...,...,...,...,...,...,...,...
2325350,1610612766,101107,-201.4262,32.7743,0.00000,0.02,NaN,4,21500492,549,8.02743,45.14262
2325351,1610612766,201587,-6.7787,359.5000,0.00000,0.02,NaN,4,21500492,549,53.30000,24.32213
2325352,1610612766,202689,58.3884,186.1024,0.00000,0.02,NaN,4,21500492,549,23.36024,19.16116
2325353,1610612766,203469,18.2134,398.0981,0.00000,0.02,NaN,4,21500492,549,44.55981,23.17866


In [15]:
def preprocess_event(data, event_id):
    event_data = data[data['event_id'] == event_id]
    
    time_step = event_data['game_clock'].value_counts().index
    for t in time_step:
        time_t_data = event_data[event_data['game_clock'] == t]

,team_id,player_id,x_loc,y_loc,radius,game_clock,shot_clock,quarter,game_id,event_id,x_loc_original,y_loc_original
0,-1,-1,-201.2734,186.7540,3.64299,711.26,11.99,1,21500492,1,23.42540,45.12734
1,1610612761,2449,110.8853,143.3811,0.00000,711.26,11.99,1,21500492,1,19.08811,13.91147
2,1610612761,201960,114.5297,53.6935,0.00000,711.26,11.99,1,21500492,1,10.11935,13.54703
3,1610612761,200768,-195.1006,160.6838,0.00000,711.26,11.99,1,21500492,1,20.81838,44.51006
4,1610612761,201942,-137.9570,94.1535,0.00000,711.26,11.99,1,21500492,1,14.16535,38.79570
...,...,...,...,...,...,...,...,...,...,...,...,...
2276362,1610612766,101107,-8.4081,219.1371,0.00000,56.39,11.99,4,21500492,527,67.33629,24.15919
2276363,1610612766,201587,64.4405,33.1574,0.00000,56.39,11.99,4,21500492,527,85.93426,31.44405
2276364,1610612766,202689,92.1229,105.4757,0.00000,56.39,11.99,4,21500492,527,78.70243,34.21229
2276365,1610612766,203469,-91.0644,-27.9717,0.00000,56.39,11.99,4,21500492,527,92.04717,15.89356


In [178]:
def create_team_data_at_time_t(time_t_team_data, event_id, team_id, time, ball_x_loc, ball_y_loc):
    column_names = ['event_id', 'team_id', 'game_clock', 'ball_x_loc', 'ball_y_loc']
    row_data = [event_id, team_id, time, ball_x_loc, ball_y_loc]
    player_ids = list(time_t_team_data['player_id'].value_counts().index)
    for i in range(1, len(player_ids) + 1):
        player_id = int(player_ids[i - 1])
        x_loc = float(time_t_team_data[time_t_team_data['player_id'] == player_id]['x_loc_original'])
        y_loc = float(time_t_team_data[time_t_team_data['player_id'] == player_id]['y_loc_original'])

        column_player = f"player{i}"
        column_x = f"player{i}_x_loc"
        column_y = f"player{i}_y_loc"

        column_names += [column_player, column_x, column_y]
        row_data += [player_id, x_loc, y_loc]

    new_t_data = pd.DataFrame(columns = column_names)
    new_t_data.loc[0] = row_data
    return new_t_data

In [244]:
event_id = 1

event_data = data[data['event_id'] == event_id]
time_step = event_data['game_clock'].value_counts().index

cleaned_event_data = None
for time in time_step:
    time_t_data = event_data[event_data['game_clock'] == time]
    time_t_team1_data = time_t_data[time_t_data['team_id'] == 1610612761]
    time_t_team2_data = time_t_data[time_t_data['team_id'] == 1610612766]
    
    time_t_ball_data = time_t_data[time_t_data['team_id'] == -1]
    ball_x_loc = float(time_t_ball_data['x_loc_original'])
    ball_y_loc = float(time_t_ball_data['y_loc_original'])
    
    a1 = create_team_data_at_time_t(time_t_team1_data, event_id, 1610612761, time, ball_x_loc, ball_y_loc)
    a2 = create_team_data_at_time_t(time_t_team2_data, event_id, 1610612766, time, ball_x_loc, ball_y_loc)
    time_t_data = pd.concat([a1, a2])
    
    if cleaned_event_data is None:
        cleaned_event_data = time_t_data
    else:
        cleaned_event_data = pd.concat([cleaned_event_data, time_t_data], ignore_index = True)
        
    cleaned_event_data = cleaned_event_data.replace({'player1': player_dict, 
                                                        'player2': player_dict, 
                                                        'player3': player_dict, 
                                                        'player4': player_dict, 
                                                        'player5': player_dict,
                                                        'team_id': team_dict})

In [209]:
cleaned_event_data['ball_x_loc'].value_counts()

23.42540    2
30.36251    2
29.81833    2
24.62517    2
25.25442    2
           ..
2.44574     2
2.38213     2
2.32867     2
2.29182     2
2.28543     2
Name: ball_x_loc, Length: 150, dtype: int64

In [238]:
event_data = pd.read_csv("/Users/yhschan/Desktop/sports analytics/Basketball-case-study-by-HMM/basketball/data/CHA-TOR-events.csv")

In [188]:
team = list(event_data[['PLAYER1_TEAM_ID', 'PLAYER1_TEAM_ABBREVIATION']].value_counts().index)
team_dict = {}
team_dict[team[0][0]] = team[0][1]
team_dict[team[1][0]] = team[1][1]
team_dict

{1610612761.0: 'TOR', 1610612766.0: 'CHA'}

In [161]:
event_data

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION
0,21500492,0,12,0,1,7:41 PM,12:00,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
1,21500492,1,10,0,1,7:41 PM,12:00,Jump Ball Valanciunas vs. Zeller: Tip to Lowry,NaN,NaN,...,Charlotte,Hornets,CHA,4,200768,Kyle Lowry,1.610613e+09,Toronto,Raptors,TOR
2,21500492,2,2,101,1,7:42 PM,11:41,MISS Scola 5' Driving Floating Jump Shot,NaN,Zeller BLOCK (1 BLK),...,NaN,NaN,NaN,5,203469,Cody Zeller,1.610613e+09,Charlotte,Hornets,CHA
3,21500492,3,4,0,1,7:42 PM,11:39,Valanciunas REBOUND (Off:1 Def:0),NaN,NaN,...,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
4,21500492,4,1,87,1,7:42 PM,11:35,Valanciunas Putback Dunk (2 PTS),NaN,NaN,...,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,21500492,543,3,11,4,9:49 PM,0:16,Lowry Free Throw 1 of 2 (17 PTS),NaN,NaN,...,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
444,21500492,545,3,12,4,9:50 PM,0:16,Lowry Free Throw 2 of 2 (18 PTS),NaN,NaN,...,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
445,21500492,548,2,1,4,9:50 PM,0:06,NaN,NaN,MISS Hairston 25' 3PT Jump Shot,...,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
446,21500492,549,4,0,4,9:50 PM,0:04,Ross REBOUND (Off:1 Def:5),NaN,NaN,...,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN


In [163]:
player_dict = {}
for i in range(len(list(a))):
    player_id, player_name = list(a)[i]
    player_dict[player_id] = player_name

In [164]:
player_dict

{202689: 'Kemba Walker',
 201960: 'DeMarre Carroll',
 1626163: 'Frank Kaminsky',
 203082: 'Terrence Ross',
 201587: 'Nicolas Batum',
 203148: 'Brian Roberts',
 203087: 'Jeremy Lamb',
 201949: 'James Johnson',
 203469: 'Cody Zeller',
 61: 'Carl Herrera',
 201942: 'DeMar DeRozan',
 201946: 'Tyler Hansbrough',
 101107: 'Marvin Williams',
 202709: 'Cory Joseph',
 202335: 'Patrick Patterson',
 202685: 'Jonas Valanciunas',
 202687: 'Bismack Biyombo',
 200768: 'Kyle Lowry',
 203798: 'PJ Hairston',
 2449: 'Luis Scola'}

In [245]:
cleaned_event_data

,event_id,team_id,game_clock,ball_x_loc,ball_y_loc,player1,player1_x_loc,player1_y_loc,player2,player2_x_loc,player2_y_loc,player3,player3_x_loc,player3_y_loc,player4,player4_x_loc,player4_y_loc,player5,player5_x_loc,player5_y_loc
0,1.0,TOR,711.26,23.42540,45.12734,Luis Scola,19.08811,13.91147,DeMarre Carroll,10.11935,13.54703,Kyle Lowry,20.81838,44.51006,DeMar DeRozan,14.16535,38.79570,Jonas Valanciunas,20.89057,34.20938
1,1.0,CHA,711.26,23.42540,45.12734,Marvin Williams,13.30744,22.19981,Nicolas Batum,9.14040,18.58649,Kemba Walker,19.31103,42.47285,Cody Zeller,16.44724,32.53167,PJ Hairston,15.49183,37.70220
2,1.0,TOR,707.50,16.83460,17.26464,Luis Scola,6.21242,5.10780,DeMarre Carroll,2.97895,25.46475,Kyle Lowry,33.80263,29.78810,DeMar DeRozan,16.66124,15.76858,Jonas Valanciunas,16.05469,34.16180
3,1.0,CHA,707.50,16.83460,17.26464,Marvin Williams,10.87066,18.69898,Nicolas Batum,5.00450,25.32575,Kemba Walker,21.17572,22.07226,Cody Zeller,13.41003,32.91308,PJ Hairston,15.08917,14.08223
4,1.0,TOR,707.42,16.23927,17.82597,Luis Scola,5.92229,4.75547,DeMarre Carroll,2.84842,26.25326,Kyle Lowry,33.41101,30.62159,DeMar DeRozan,16.35211,16.47311,Jonas Valanciunas,15.86899,34.01660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1.0,CHA,709.18,34.07072,22.66508,Marvin Williams,12.50838,19.06709,Nicolas Batum,9.46248,22.40999,Kemba Walker,29.70235,24.77964,Cody Zeller,16.33376,35.29623,PJ Hairston,4.58773,17.35987
296,1.0,TOR,709.14,33.00436,21.20693,Luis Scola,12.18967,12.53035,DeMarre Carroll,6.17195,16.88200,Kyle Lowry,34.15038,22.51113,DeMar DeRozan,7.06284,10.96900,Jonas Valanciunas,22.52191,37.10673
297,1.0,CHA,709.14,33.00436,21.20693,Marvin Williams,12.42154,19.04711,Nicolas Batum,9.48987,22.45474,Kemba Walker,29.69889,24.53432,Cody Zeller,16.26223,35.27820,PJ Hairston,4.66247,16.70776
298,1.0,TOR,705.30,2.28543,4.39596,Luis Scola,5.62772,5.87473,DeMarre Carroll,5.73374,43.28594,Kyle Lowry,29.07003,39.29628,DeMar DeRozan,22.24552,21.64748,Jonas Valanciunas,9.67097,29.11568


In [217]:
import math
pre_time_step_data = cleaned_event_data.iloc[0]
cur_time_step_data = cleaned_event_data.iloc[2]

time_diff = pre_time_step_data['game_clock'] - cur_time_step_data['game_clock']
ball_dist = math.sqrt(
                    (pre_time_step_data['ball_x_loc'] - cur_time_step_data['ball_x_loc'])**2 + 
                    (pre_time_step_data['ball_y_loc'] - cur_time_step_data['ball_y_loc'])**2
                    )


In [218]:
time_diff

3.759999999999991

In [206]:
pre_time_step_data['ball_x_loc'] - cur_time_step_data['ball_x_loc']

0.0

In [246]:
def preoprocess_speed(cleaned_event_data):
        # Divide by two because each time step gets 2 team data, so it should be an even number
        n_timestep = len(cleaned_event_data)

        ball_speed_list = []
        player_speed_list_nested = [[], [], [], [], []]

        for index in range(n_timestep):
        # Set value to 0 if it is the beginning of the event
            if index == 0 or index == 1:
                ball_speed_list += [0]
                for j in range(1, 6):
                    player_speed_list_nested[j-1] += [0] 
                continue
            
            # Prepare previous time step and current time step data
            pre_time_step_data = cleaned_event_data.iloc[index - 2]
            cur_time_step_data = cleaned_event_data.iloc[index]

            time_diff = pre_time_step_data['game_clock'] - cur_time_step_data['game_clock']

            # Calculate ball speed
            ball_speed = math.sqrt(
                                    (pre_time_step_data['ball_x_loc'] - cur_time_step_data['ball_x_loc'])**2 + 
                                    (pre_time_step_data['ball_y_loc'] - cur_time_step_data['ball_y_loc'])**2
                                    ) / time_diff
            ball_speed_list += [ball_speed]

            # Calculate player speed
            for j in range(1, 6):
                x_name, y_name = f"player{j}_x_loc", f"player{j}_y_loc"
                player_speed = math.sqrt(
                                    (pre_time_step_data[x_name] - cur_time_step_data[x_name])**2 + 
                                    (pre_time_step_data[y_name] - cur_time_step_data[y_name])**2
                                    ) / time_diff

                player_speed_list_nested[j-1] += [player_speed] 

        # Add all speed data to the original cleaned dataset
        print(len(ball_speed_list))
        print(len(player_speed_list_nested[0]))
        cleaned_event_data['ball_speed'] = ball_speed_list
        for j in range(1, 6):
            speed_name = f"player{j}_speed"
            cleaned_event_data[speed_name] = player_speed_list_nested[j-1]

        return cleaned_event_data      

In [222]:
len(cleaned_event_data)/2

150.0

In [247]:
new_data = preoprocess_speed(cleaned_event_data)
new_data

300
300


,event_id,team_id,game_clock,ball_x_loc,ball_y_loc,player1,player1_x_loc,player1_y_loc,player2,player2_x_loc,...,player4_y_loc,player5,player5_x_loc,player5_y_loc,ball_speed,player1_speed,player2_speed,player3_speed,player4_speed,player5_speed
0,1.0,TOR,711.26,23.42540,45.12734,Luis Scola,19.08811,13.91147,DeMarre Carroll,10.11935,...,38.79570,Jonas Valanciunas,20.89057,34.20938,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1.0,CHA,711.26,23.42540,45.12734,Marvin Williams,13.30744,22.19981,Nicolas Batum,9.14040,...,32.53167,PJ Hairston,15.49183,37.70220,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,1.0,TOR,707.50,16.83460,17.26464,Luis Scola,6.21242,5.10780,DeMarre Carroll,2.97895,...,15.76858,Jonas Valanciunas,16.05469,34.16180,7.614788,4.148322,3.694965,5.220677,6.160103,1.286201
3,1.0,CHA,707.50,16.83460,17.26464,Marvin Williams,10.87066,18.69898,Nicolas Batum,5.00450,...,32.91308,PJ Hairston,15.08917,14.08223,7.614788,1.134417,2.102970,5.448307,0.814113,6.282820
4,1.0,TOR,707.42,16.23927,17.82597,Luis Scola,5.92229,4.75547,DeMarre Carroll,2.84842,...,16.47311,Jonas Valanciunas,15.86899,34.01660,10.227943,5.705149,9.990512,11.511352,9.617074,2.946596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1.0,CHA,709.18,34.07072,22.66508,Marvin Williams,12.50838,19.06709,Nicolas Batum,9.46248,...,35.29623,PJ Hairston,4.58773,17.35987,6.041360,2.023423,1.742117,6.185278,1.884300,16.372846
296,1.0,TOR,709.14,33.00436,21.20693,Luis Scola,12.18967,12.53035,DeMarre Carroll,6.17195,...,10.96900,Jonas Valanciunas,22.52191,37.10673,45.161689,4.881841,2.017615,6.875061,15.767855,5.828509
297,1.0,CHA,709.14,33.00436,21.20693,Marvin Williams,12.42154,19.04711,Nicolas Batum,9.48987,...,35.27820,PJ Hairston,4.66247,16.70776,45.161689,2.227721,1.311672,6.133610,1.844184,16.409477
298,1.0,TOR,705.30,2.28543,4.39596,Luis Scola,5.62772,5.87473,DeMarre Carroll,5.73374,...,21.64748,Jonas Valanciunas,9.67097,29.11568,9.119275,2.433976,6.876973,4.566963,4.833825,3.940850


In [239]:
event_data

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION
0,21500492,0,12,0,1,7:41 PM,12:00,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
1,21500492,1,10,0,1,7:41 PM,12:00,Jump Ball Valanciunas vs. Zeller: Tip to Lowry,NaN,NaN,...,Charlotte,Hornets,CHA,4,200768,Kyle Lowry,1.610613e+09,Toronto,Raptors,TOR
2,21500492,2,2,101,1,7:42 PM,11:41,MISS Scola 5' Driving Floating Jump Shot,NaN,Zeller BLOCK (1 BLK),...,NaN,NaN,NaN,5,203469,Cody Zeller,1.610613e+09,Charlotte,Hornets,CHA
3,21500492,3,4,0,1,7:42 PM,11:39,Valanciunas REBOUND (Off:1 Def:0),NaN,NaN,...,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
4,21500492,4,1,87,1,7:42 PM,11:35,Valanciunas Putback Dunk (2 PTS),NaN,NaN,...,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,21500492,543,3,11,4,9:49 PM,0:16,Lowry Free Throw 1 of 2 (17 PTS),NaN,NaN,...,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
444,21500492,545,3,12,4,9:50 PM,0:16,Lowry Free Throw 2 of 2 (18 PTS),NaN,NaN,...,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
445,21500492,548,2,1,4,9:50 PM,0:06,NaN,NaN,MISS Hairston 25' 3PT Jump Shot,...,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
446,21500492,549,4,0,4,9:50 PM,0:04,Ross REBOUND (Off:1 Def:5),NaN,NaN,...,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN


In [259]:
cleaned_event_data.sort_values(by = 'game_clock', ascending = False)

,event_id,team_id,game_clock,ball_x_loc,ball_y_loc,player1,player1_x_loc,player1_y_loc,player2,player2_x_loc,...,player5,player5_x_loc,player5_y_loc,ball_speed,player1_speed,player2_speed,player3_speed,player4_speed,player5_speed,index
index,,,,,,,,,,,,,,,,,,,,,
0,1.0,TOR,711.26,23.42540,45.12734,Luis Scola,19.08811,13.91147,DeMarre Carroll,10.11935,...,Jonas Valanciunas,20.89057,34.20938,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
1,1.0,CHA,711.26,23.42540,45.12734,Marvin Williams,13.30744,22.19981,Nicolas Batum,9.14040,...,PJ Hairston,15.49183,37.70220,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1
76,1.0,TOR,711.22,24.00450,44.68890,Luis Scola,18.93523,13.82250,DeMarre Carroll,9.98343,...,Jonas Valanciunas,20.99513,34.48187,-10.024955,-3.854498,-4.797212,-2.557251,-4.050047,-1.704459,76
77,1.0,CHA,711.22,24.00450,44.68890,Marvin Williams,13.12847,22.14854,Nicolas Batum,8.97947,...,PJ Hairston,15.07591,37.49843,-10.024955,-1.480668,-2.672663,-4.171983,-1.391924,-4.188140,77
195,1.0,CHA,711.18,24.62517,44.30089,Marvin Williams,12.96599,22.08291,Nicolas Batum,8.82831,...,PJ Hairston,14.64087,37.25561,-12.608348,-3.492018,-2.730416,-11.470383,-2.653726,-10.356748,195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,1.0,CHA,705.38,2.29182,4.00409,Marvin Williams,6.24678,4.65138,Nicolas Batum,4.40007,...,PJ Hairston,14.14589,23.17964,2.268486,3.759811,0.259601,1.210109,3.701265,2.013664,107
108,1.0,TOR,705.34,2.27810,4.11382,Luis Scola,5.70868,5.58918,DeMarre Carroll,5.64141,...,Jonas Valanciunas,9.75277,29.14539,2.764610,4.999850,4.381694,0.622502,9.091575,2.705844,108
109,1.0,CHA,705.34,2.27810,4.11382,Marvin Williams,6.25324,4.86672,Nicolas Batum,4.40300,...,PJ Hairston,14.16046,23.11583,2.764610,5.385922,0.468018,1.206842,4.149394,1.636307,109
